In [11]:
# ===========================================
# PARAMÈTRES DE CONFIGURATION
# ===========================================

# Durée d'analyse (en jours)
DUREE_JOURS = 5 * 365

# Intervalle des données
# Options: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
INTERVAL = '1d'

# Période de la moyenne mobile (en nombre de périodes)
MA_PERIODE = 5

print(f"Configuration:")
print(f"- Durée d'analyse: {DUREE_JOURS} jours")
print(f"- Intervalle: {INTERVAL}")
print(f"- Moyenne mobile: {MA_PERIODE} périodes")
print()

# ===========================================
# RÉCUPÉRATION DES DONNÉES
# ===========================================

import pandas as pd
import requests
from datetime import datetime
import time

print("Récupération des données Bitcoin...")

# Fonction pour récupérer des données par batch
def get_historical_data(symbol='BTCUSDT', interval=INTERVAL, start_time=None, days_back=DUREE_JOURS):
    """
    Récupère les données selon les paramètres configurés
    """
    all_data = []
    limit = 1000  # Maximum autorisé par Binance
    
    # Date de début selon la durée configurée
    if start_time is None:
        start_time = int((datetime.now() - pd.Timedelta(days=days_back)).timestamp() * 1000)
    
    current_time = start_time
    end_time = int(datetime.now().timestamp() * 1000)
    
    # Calcul de l'incrément temporel selon l'intervalle
    interval_ms = {
        '1m': 60 * 1000,
        '3m': 3 * 60 * 1000,
        '5m': 5 * 60 * 1000,
        '15m': 15 * 60 * 1000,
        '30m': 30 * 60 * 1000,
        '1h': 60 * 60 * 1000,
        '2h': 2 * 60 * 60 * 1000,
        '4h': 4 * 60 * 60 * 1000,
        '6h': 6 * 60 * 60 * 1000,
        '8h': 8 * 60 * 60 * 1000,
        '12h': 12 * 60 * 60 * 1000,
        '1d': 24 * 60 * 60 * 1000,
        '3d': 3 * 24 * 60 * 60 * 1000,
        '1w': 7 * 24 * 60 * 60 * 1000,
        '1M': 30 * 24 * 60 * 60 * 1000
    }
    
    time_increment = interval_ms.get(interval, 60 * 1000)  # Défaut: 1 minute
    
    batch_count = 0
    while current_time < end_time:
        batch_count += 1
        print(f"Batch {batch_count} ({symbol}): Récupération depuis {datetime.fromtimestamp(current_time/1000).strftime('%Y-%m-%d %H:%M')}")
        
        # Construction de l'URL avec startTime
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={current_time}"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
                
            all_data.extend(data)
            
            # Mettre à jour current_time avec le timestamp de la dernière bougie + intervalle
            last_timestamp = data[-1][0]
            current_time = last_timestamp + time_increment
            
            # Pause pour respecter les limites de taux
            time.sleep(0.2)
            
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête: {e}")
            break
    
    print(f"Total de {len(all_data)} périodes ({interval}) récupérées pour {symbol}")
    return all_data

# Récupération des données Bitcoin selon les paramètres configurés
print(f"=== Récupération Bitcoin (BTCUSDT) - Intervalle {INTERVAL} ({DUREE_JOURS} jours) ===")
btc_response = get_historical_data('BTCUSDT', interval=INTERVAL, days_back=DUREE_JOURS)

# Fonction pour créer un DataFrame
def create_dataframe(data, symbol_name):
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    # Affichage des informations adaptées à l'intervalle
    print(f"\nDonnées {symbol_name} récupérées:")
    print(f"Période: {df.index.min().strftime('%Y-%m-%d %H:%M')} à {df.index.max().strftime('%Y-%m-%d %H:%M')}")
    print(f"Nombre de périodes ({INTERVAL}): {len(df)}")
    
    # Calculs selon l'intervalle
    if INTERVAL.endswith('m'):
        minutes_per_period = int(INTERVAL[:-1])
        hours = len(df) * minutes_per_period / 60
        days = hours / 24
        print(f"Nombre d'heures: {hours:.1f}")
        print(f"Nombre de jours: {days:.1f}")
    elif INTERVAL.endswith('h'):
        hours_per_period = int(INTERVAL[:-1])
        hours = len(df) * hours_per_period
        days = hours / 24
        print(f"Nombre d'heures: {hours:.1f}")
        print(f"Nombre de jours: {days:.1f}")
    elif INTERVAL == '1d':
        print(f"Nombre de jours: {len(df)}")
    
    print(f"Prix minimum: ${df['low'].min():.2f}")
    print(f"Prix maximum: ${df['high'].max():.2f}")
    
    return df

# Création du DataFrame Bitcoin
df_btc = create_dataframe(btc_response, "Bitcoin")

print(f"\nPremières lignes Bitcoin (intervalle {INTERVAL}):")
print(df_btc.head())

Configuration:
- Durée d'analyse: 1825 jours
- Intervalle: 1d
- Moyenne mobile: 5 périodes

Récupération des données Bitcoin...
=== Récupération Bitcoin (BTCUSDT) - Intervalle 1d (1825 jours) ===
Batch 1 (BTCUSDT): Récupération depuis 2020-09-04 22:03
Batch 2 (BTCUSDT): Récupération depuis 2023-06-02 02:00
Total de 1825 périodes (1d) récupérées pour BTCUSDT

Données Bitcoin récupérées:
Période: 2020-09-05 00:00 à 2025-09-03 00:00
Nombre de périodes (1d): 1825
Nombre de jours: 1825
Prix minimum: $9825.00
Prix maximum: $124474.00

Premières lignes Bitcoin (intervalle 1d):
                open      high      low     close        volume  \
timestamp                                                         
2020-09-05  10446.25  10565.68  9825.00  10166.69  90001.605568   
2020-09-06  10166.69  10347.14  9994.86  10256.20  56368.788815   
2020-09-07  10255.89  10410.75  9875.00  10373.44  62620.230676   
2020-09-08  10373.45  10438.00  9850.00  10126.65  73491.878418   
2020-09-09  10126.66 

In [12]:
# Graphique Bitcoin avec volumes et moyenne mobile
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calcul de la moyenne mobile selon le paramètre configuré
df_btc[f'ma_{MA_PERIODE}'] = df_btc['close'].rolling(window=MA_PERIODE).mean()

# Création de sous-graphiques pour Bitcoin
fig_btc = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    
    row_heights=[0.7, 0.3],
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}]]
)

# Graphique en chandeliers Bitcoin (prix)
fig_btc.add_trace(
    go.Candlestick(
        x=df_btc.index,
        open=df_btc['open'],
        high=df_btc['high'],
        low=df_btc['low'],
        close=df_btc['close'],
        name="Bitcoin",
        increasing_line_color='#f7931a',  # Orange Bitcoin
        decreasing_line_color='#d32f2f'
    ), 
    row=1, col=1
)

# Ajout de la moyenne mobile selon le paramètre
fig_btc.add_trace(
    go.Scatter(
        x=df_btc.index,
        y=df_btc[f'ma_{MA_PERIODE}'],
        mode='lines',
        name=f'MM {MA_PERIODE} périodes',
        line=dict(color='blue', width=2),
        opacity=0.8
    ),
    row=1, col=1
)

# Graphique des volumes Bitcoin
fig_btc.add_trace(
    go.Bar(
        x=df_btc.index,
        y=df_btc['volume'],
        name="Volume BTC",
        marker_color='rgba(247, 147, 26, 0.7)',  # Orange Bitcoin transparent
        marker_line_color='rgba(247, 147, 26, 0.8)',
        marker_line_width=0.5
    ), 
    row=2, col=1
)

# Configuration du layout Bitcoin
fig_btc.update_layout(
    title={
        'text': f"Bitcoin (BTC/USDT) avec Moyenne Mobile {MA_PERIODE} périodes",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    height=800,
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    ),
    xaxis_rangeslider_visible=False,
    template="plotly_white",
    margin=dict(l=50, r=50, t=80, b=50)
)

# Configuration des axes Bitcoin
fig_btc.update_yaxes(title_text="Prix (USD)", row=1, col=1)
fig_btc.update_yaxes(title_text="Volume", row=2, col=1)

# Affichage avec configuration explicite
config = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
}

print(f"Affichage du graphique Bitcoin avec moyenne mobile {MA_PERIODE} périodes...")
print(f"Moyenne mobile calculée sur {len(df_btc[~df_btc[f'ma_{MA_PERIODE}'].isna()])} périodes (NaN pour les {MA_PERIODE} premières périodes)")
fig_btc.show(config=config)

Affichage du graphique Bitcoin avec moyenne mobile 5 périodes...
Moyenne mobile calculée sur 1821 périodes (NaN pour les 5 premières périodes)
